# Batch run models

In [ ]:
%load_ext lab_black
import os, multiprocessing, time, papermill, sys
from IPython.display import clear_output

sys.path.append("/home/jupyter/tf/src")
from meta import ModelConfig, make_batch_cfg, parse_batch_results

# nest_asyncio.apply()

## Make configs

In [ ]:
batch_name = "sampling_proper"
batch_output_dir = f"models/batch_run/{batch_name}/"

# Gen OG random seed: [int(random.random() * 1e5) for x in range(10)]

param_grid = {
    "sample_name": ["chang", "experimental", "hs04", "jay", "wf_linear_cutoff"],
}

static_hpar = {
    "rng_seed": 53797,
    "p_noise": 0.0,
    "zero_error_radius": 0.1,
    "cleanup_units": 50,
    "use_semantic": False,
    "input_dim": 119,
    "output_dim": 250,
    "hidden_units": 100,
    "pretrain_attractor": False,
    "rnn_activation": "sigmoid",
    "w_initializer": 0.1,
    "regularizer_const": None,
    "tau": 1 / 3,
    "max_unit_time": 4.0,
    "output_ticks": 2,
    "optimizer": "adam",
    "learning_rate": 0.005,
    "n_mil_sample": 1.0,
    "batch_size": 128,
    "save_freq": 10,
    "batch_name": batch_name,
    "show_plots_in_notebook": True,
}

batch_cfgs = make_batch_cfg(
    batch_name, batch_output_dir, static_hpar, param_grid, "OSP_master.ipynb"
)

## Parallel run
- Can run on Jupyter Lab
- Cannot run on VSCode

In [ ]:


# Run
def run_batch(cfg):
    """
    Using papermill to run parameterized notebook
    To prevent overwriting, set default overwrite to False if needed
    """

    print("Running model {}".format(cfg["sn"]))
    os.makedirs(cfg["model_folder"], exist_ok=True)
    papermill.execute_notebook(
        cfg["in_notebook"],
        cfg["out_notebook"],
        parameters=cfg["params"],
        prepare_only=True
    )
    
    in_notebook = cfg["out_notebook"]
    out_dir = cfg["model_folder"]
    !jupyter nbconvert --ExecutePreprocessor.timeout=12000 --output-dir $out_dir --to notebook --execute $in_notebook
    !jupyter nbconvert --output-dir $out_dir --to html_toc $in_notebook
    clear_output()


# Run in parallel pool
# with multiprocessing.Pool(2) as pool:
#     pool.map(run_batch, batch_cfgs)

[run_batch(cfg) for cfg in batch_cfgs]
print("Done")

Compile and save results

In [ ]:
# Cannot execute together with parallel run for some reason... maybe due to nest_asynio... forgot how to fix...
cfgs, df = parse_batch_results(batch_cfgs)
df.to_csv(batch_output_dir + "bcdf.csv")
cfgs.to_csv(batch_output_dir + "cfgs.csv")

Shutdown compute engine

In [ ]:
time.sleep(30)
print('shuting down')
!sudo poweroff

In [ ]:
!jupyter nbconvert --ExecutePreprocessor.timeout=6000 --to html_toc --execute OSP_master.ipynb --output-dir $cfg.path_model_folder